In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

### 
testplayer='203076'
testgame = '0022201200'
testteam = '1610612755'

In [6]:
## MASTER CREATION, should complete this first

def process_endpoint(conn, endpoint_name, endpoint_function, params):
    for param in params:
        response = endpoint_function(param).get_data_frames()  # Call the endpoint
        for index, dataframe in enumerate(response):
            table_name = f"{endpoint_name}_{index}"
            create_endpoint_table(conn, endpoint_name, dataframe, index)
            insert_endpoint_data(conn, table_name, dataframe)



In [2]:
##### THIS IS A TABLE UPDATE, SPECIFICALLY FOR GAME RELATED TABLES, DOES NOT UPDATE PLAYER TABLES
### TO BE INCLUDED IN THE ARGUMENT param (team or player or game (or any combination)) , endpoint, conn


######
## All of this needs figured out
######



## adds the connection
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

## Naming stuff
index = 0
param = testgame

list(nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3.expected_data.keys())[0]

# need to replace with the endpoint class
namepre = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3.__name__.lower()
name = namepre+"_"+str(index)

df = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3(param).get_data_frames()[index]

df = allintwo.clean_column_names(df)


if not allintwo.check_table_exists(conn,name):
    allintwo.create_table(conn,name,df)
    ## Need to put the back fill process here

#returns all games that aren't currently in the table
gameslist = allintwo.game_difference(conn,name)



Connected to RDS PostgreSQL database


In [4]:
for i in gameslist:
    tdf = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3(param).get_data_frames()[index]
    allintwo.insert_dataframe_to_rds(conn,tdf,name)
    time.sleep(2)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [10]:
name

'boxscoreadvancedv3_0'

In [5]:
df = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=testgame).get_data_frames()[0]


In [13]:
', '.join([f"{col} {(dtype)}" for col, dtype in zip(df.columns, df.dtypes)])

'gameid object, teamid int64, teamcity object, teamname object, teamtricode object, teamslug object, personid int64, firstname object, familyname object, namei object, playerslug object, position object, comment object, jerseynum object, minutes object, estimatedoffensiverating float64, offensiverating float64, estimateddefensiverating float64, defensiverating float64, estimatednetrating float64, netrating float64, assistpercentage float64, assisttoturnover float64, assistratio float64, offensivereboundpercentage float64, defensivereboundpercentage float64, reboundpercentage float64, turnoverratio float64, effectivefieldgoalpercentage float64, trueshootingpercentage float64, usagepercentage float64, estimatedusagepercentage float64, estimatedpace float64, pace float64, paceper40 float64, possessions float64, pie float64'

In [14]:
df

,gameid,teamid,teamcity,teamname,teamtricode,teamslug,personid,firstname,familyname,namei,...,turnoverratio,effectivefieldgoalpercentage,trueshootingpercentage,usagepercentage,estimatedusagepercentage,estimatedpace,pace,paceper40,possessions,pie
0,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,18.2,0.143,0.285,0.200,0.202,109.96,107.09,89.24,42.0,0.000
1,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1631114,Jalen,Williams,J. Williams,...,11.8,0.450,0.450,0.164,0.167,107.05,106.95,89.13,62.0,0.030
2,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1631119,Jaylin,Williams,J. Williams,...,9.1,0.688,0.688,0.143,0.144,110.21,108.05,90.04,51.0,0.107
3,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1630581,Josh,Giddey,J. Giddey,...,14.3,0.500,0.539,0.215,0.218,105.01,105.55,87.96,78.0,0.117
4,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1628983,Shai,Gilgeous-Alexander,S. Gilgeous-Alexander,...,3.1,0.395,0.470,0.243,0.248,105.60,104.99,87.49,83.0,0.117
5,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1630598,Aaron,Wiggins,A. Wiggins,...,0.0,0.611,0.697,0.157,0.164,101.58,104.79,87.32,62.0,0.175
6,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1630198,Isaiah,Joe,I. Joe,...,11.8,0.409,0.463,0.264,0.272,100.71,101.57,84.64,45.0,0.133
7,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,203967,Dario,Šarić,D. Šarić,...,0.0,0.600,0.643,0.196,0.204,97.44,100.09,83.40,49.0,0.262
8,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1630322,Lindy,Waters III,L. Waters III,...,0.0,0.714,0.714,0.167,0.183,97.99,104.15,86.79,36.0,0.181
9,0022201200,1610612760,Oklahoma City,Thunder,OKC,thunder,1630544,Tre,Mann,T. Mann,...,0.0,0.000,0.000,0.000,0.000,75.00,90.00,75.00,3.0,0.000


'boxscoreadvancedv3_0'

In [5]:
leaguegamefinder.LeagueGameFinder(date_to_nullable='10/30/2024',date_from_nullable='10/28/2024').get_data_frames()[0]


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612741,CHI,Chicago Bulls,0022400123,2024-10-30,CHI vs. ORL,W,239,102,...,0.680,9,45,54,23,6,2,14,16,3.0
1,22024,1610612738,BOS,Boston Celtics,0022400119,2024-10-30,BOS @ IND,L,266,132,...,0.806,18,33,51,23,11,5,14,19,-3.0
2,22024,1610612748,MIA,Miami Heat,0022400122,2024-10-30,MIA vs. NYK,L,239,107,...,0.667,7,32,39,27,6,3,10,17,-9.0
3,22024,1610612764,WAS,Washington Wizards,0022400121,2024-10-30,WAS vs. ATL,W,242,133,...,0.920,6,31,37,32,10,5,14,21,13.0
4,22024,1610612760,OKC,Oklahoma City Thunder,0022400125,2024-10-30,OKC vs. SAS,W,242,105,...,0.750,5,34,39,28,18,6,10,15,12.0
5,22024,1610612753,ORL,Orlando Magic,0022400123,2024-10-30,ORL @ CHI,L,239,99,...,0.900,6,36,42,25,9,8,12,19,-3.0
6,22024,1610612755,PHI,Philadelphia 76ers,0022400120,2024-10-30,PHI vs. DET,L,241,95,...,0.750,8,29,37,17,7,4,9,21,-10.0
7,22024,1610612751,BKN,Brooklyn Nets,0022400124,2024-10-30,BKN @ MEM,W,240,119,...,0.880,8,26,34,27,10,2,15,25,13.0
8,22024,1610612761,TOR,Toronto Raptors,0022400117,2024-10-30,TOR @ CHA,L,240,133,...,0.667,13,28,41,37,9,5,14,25,-5.0
9,22024,1610612766,CHA,Charlotte Hornets,0022400117,2024-10-30,CHA vs. TOR,W,239,138,...,0.800,14,24,38,31,9,8,15,24,5.0


In [42]:
nbaapi.playergamelogs.PlayerGameLogs(date_from_nullable='10/20/2000',
    date_to_nullable='12/31/2001',  
    season_nullable='2000-01').get_data_frames()[0].sort_values('GAME_DATE')

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,MIN_SEC
23972,2000-01,251,Sean Elliott,Sean,1610612759,SAS,San Antonio Spurs,0020000009,2000-10-31T00:00:00,SAS vs. IND,W,25.645000,3,7,0.429,2,4,0.500,2,2,1.000,0,0,0,2,0,0,0,0,4,0,10,11,13.0,0,0,14.0,1,1,1,1,11048,10068,10857,11237,1254,2058,2291,6583,8212,1,13595,19944,21122,7623,16423,11514,7539,1,18667,165,8809,2992,15064,2081,46,13486,None,25:39
23793,2000-01,673,Alan Henderson,Alan,1610612737,ATL,Atlanta Hawks,0020000004,2000-10-31T00:00:00,ATL vs. CHH,L,35.900000,9,11,0.818,0,0,0.000,5,9,0.556,1,1,2,2,4,3,1,0,4,0,23,-12,36.4,0,0,35.0,1,11963,11963,11963,4998,1294,5916,1253,6642,11355,6642,2061,872,10196,7381,15611,14531,7623,900,555,2810,1,18667,165,1611,21136,3427,2081,46,3002,None,35:54
23792,2000-01,452,Vin Baker,Vin,1610612760,SEA,Seattle SuperSonics,0020000013,2000-10-31T00:00:00,SEA @ VAN,L,37.473333,7,16,0.438,0,0,0.000,5,6,0.833,2,5,7,1,3,1,4,1,3,0,19,-3,40.9,0,0,37.0,1,11963,11963,11963,4069,2828,2449,11104,6642,11355,6642,2061,2259,6470,3870,3997,4027,11220,2182,4616,257,15185,14225,165,2930,14292,2321,2081,46,2491,None,37:28
23791,2000-01,764,David Robinson,David,1610612759,SAS,San Antonio Spurs,0020000009,2000-10-31T00:00:00,SAS vs. IND,W,31.111667,5,9,0.556,0,0,0.000,12,12,1.000,1,8,9,0,1,1,3,1,2,0,22,18,43.8,0,0,39.0,1,1,1,1,7743,5569,8143,5547,6642,11355,6642,99,279,1,7381,1166,2244,16481,9383,4616,543,15185,8838,165,1863,1031,1780,2081,46,2054,None,31:07
23790,2000-01,1517,Bobby Jackson,Bobby,1610612758,SAC,Sacramento Kings,0020000006,2000-10-31T00:00:00,SAC @ CHI,W,36.233333,5,9,0.556,1,3,0.333,3,3,1.000,0,4,4,6,1,4,0,1,4,0,14,18,38.8,0,0,33.0,1,1,1,1,4795,5569,8143,5547,2963,3416,4566,4676,7041,1,13595,5746,9016,1727,9383,172,7539,15185,18667,165,5590,1031,2829,2081,46,3606,None,36:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,2000-01,213,Antonio Davis,Antonio,1610612761,TOR,Toronto Raptors,0020001182,2001-04-18T00:00:00,TOR @ WAS,W,31.468333,5,12,0.417,0,0,0.000,0,0,0.000,2,8,10,1,1,1,0,1,3,0,10,5,25.5,1,0,23.0,1,1,1,1,7541,5569,5057,11927,6642,11355,6642,13632,14518,13632,3870,1166,1630,11220,9383,4616,7539,15185,14225,165,8809,6400,7741,1,46,7894,None,31:28
142,2000-01,1497,Chauncey Billups,Chauncey,1610612750,MIN,Minnesota Timberwolves,0020001185,2001-04-18T00:00:00,MIN @ DAL,L,19.250000,8,13,0.615,3,4,0.750,2,2,1.000,0,1,1,2,0,1,0,1,1,0,21,13,28.2,0,0,29.0,1,11963,11963,11963,15112,1945,4248,3880,451,2058,1303,6583,8212,1,13595,15611,17898,7623,16423,4616,7539,15185,3659,165,2205,2218,6459,2081,46,5050,None,19:15
141,2000-01,702,Voshon Lenard,Voshon,1610612743,DEN,Denver Nuggets,0020001186,2001-04-18T00:00:00,DEN vs. SAC,W,27.966667,5,14,0.357,3,7,0.429,0,0,0.000,0,4,4,7,3,1,0,0,2,0,13,13,28.3,0,0,29.0,1,1,1,1,9607,5569,3540,14402,451,395,4073,13632,14518,13632,13595,5746,9016,1164,2182,4616,7539,1,8838,165,6325,2218,6408,2081,46,5050,None,27:58
152,2000-01,239,Darrick Martin,Darrick,1610612758,SAC,Sacramento Kings,0020001186,2001-04-18T00:00:00,SAC @ DEN,L,21.033333,4,8,0.500,2,3,0.667,5,6,0.833,0,3,3,3,1,0,0,1,3,0,15,0,22.1,0,0,23.0,1,11963,11963,11963,14002,7605,9408,6674,1254,3416,1504,2061,2259,6470,13595,8242,11534,5270,9383,11514,7539,15185,14225,165,4959,10979,9501,2081,46,789